# 🎯 LangGraph Basics: StateGraph Fundamentals

This notebook introduces the core concepts of **LangGraph** - a framework for building stateful, multi-step AI applications.

## 🧠 What You'll Learn

1. **StateGraph** - Container for workflow logic
2. **Nodes** - Processing units that modify state
3. **Edges** - Connections between nodes (static & conditional)
4. **State** - Shared data structure that persists across nodes
5. **Conditional Routing** - Dynamic flow control based on state

## 💡 Why LangGraph?

Traditional LLM applications run linearly: Input → Process → Output

LangGraph enables:
- ✅ **Multi-step workflows** with state persistence
- ✅ **Conditional branching** based on intermediate results
- ✅ **Loops and cycles** for iterative processing
- ✅ **Human-in-the-loop** for approvals and feedback

---

## 📚 Core Concepts

### 🔹 StateGraph
The container that holds all nodes, edges, and the definition of the shared state.

### 🔹 Node
A callable function that:
- Takes current state as input
- Processes/transforms data
- Returns updates to merge into state

### 🔹 Edge
The transition rule showing which node runs after which:
- **Static Edge**: Always goes to the same next node
- **Conditional Edge**: Routes based on state values

### 🔹 State
A dictionary-like structure that:
- Persists across node executions
- Automatically merges updates from nodes
- Can be accessed by all nodes

---

## 🎯 Example: Counter with Conditional Loop

Let's build a simple counter that:
1. Increments a value
2. Checks if it should continue
3. Loops until reaching a threshold
4. Then ends

### Step 1: Import Dependencies

In [ ]:
from typing import TypedDict
from langgraph.graph import END, StateGraph

print("✅ LangGraph imported successfully!")

d:\langraph\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


{'count': 5}


### Step 2: Define State Schema

The state schema defines what data the graph will track.

In [ ]:
# Define the state structure
class SimpleState(TypedDict):
    count: int  # Our counter variable

print("✅ State schema defined: SimpleState with 'count' field")

### Step 3: Define Nodes

Nodes are functions that process and update the state.

In [ ]:
# Node: Increments the counter
def increment(state: SimpleState) -> SimpleState: 
    """
    Takes current state, increments count by 1
    Returns updated state dictionary
    """
    new_count = state["count"] + 1
    print(f"⬆️  Incrementing: {state['count']} → {new_count}")
    
    return {"count": new_count}

print("✅ Node function 'increment' defined")

### Step 4: Define Conditional Router

The router decides which edge to follow based on current state.

In [ ]:
# Router: Decides whether to continue or stop
def should_continue(state: SimpleState) -> str:
    """
    Conditional logic:
    - If count < 5: return "continue" (loop back to increment)
    - If count >= 5: return "stop" (end the graph)
    """
    if state["count"] < 5: 
        print(f"🔄 Count is {state['count']} - continuing loop")
        return "continue"
    else: 
        print(f"🛑 Count reached {state['count']} - stopping")
        return "stop"

print("✅ Router function 'should_continue' defined")

### Step 5: Build the Graph

Now we construct the graph by:
1. Creating a StateGraph instance
2. Adding nodes
3. Setting entry point
4. Adding conditional edges
5. Compiling

In [ ]:
# Create the graph
graph = StateGraph(SimpleState)

# Add the increment node
graph.add_node("increment", increment)

# Set entry point (where execution starts)
graph.set_entry_point("increment")

# Add conditional edges from increment node
# The router function decides which path to take
graph.add_conditional_edges(
    "increment",  # Source node
    should_continue,  # Router function
    {
        "continue": "increment",  # Loop back to increment
        "stop": END  # End the graph
    }
)

# Compile the graph into a runnable app
app = graph.compile()

print("✅ Graph compiled successfully!")
print("\n📊 Graph Structure:")
print("   START → increment → [continue/stop]")
print("           ↑            ↓")
print("           └─ loop ─────┘")
print("                        ↓")
print("                       END")

### Step 6: Execute the Graph

Let's run the graph with an initial state and see it loop!

In [ ]:
# Initial state
state = {"count": 0}

print("🚀 Starting graph execution with count = 0\n")
print("=" * 50)

# Run the graph
result = app.invoke(state)

print("=" * 50)
print(f"\n✅ Final Result: {result}")
print(f"🎯 Counter reached: {result['count']}")

---

## 🎓 Key Takeaways

### ✅ What Happened

1. **State initialized** with `count = 0`
2. **Node executed** - incremented count to 1
3. **Router checked** - count < 5, so returned "continue"
4. **Loop repeated** - increment node ran again
5. **Process repeated** until count reached 5
6. **Router stopped** - returned "stop" when count = 5
7. **Graph ended** with final state

### 🔑 Core Concepts Demonstrated

| Concept | Usage in Example |
|---------|------------------|
| **StateGraph** | Container for our counter workflow |
| **State (TypedDict)** | Defined `count` field |
| **Node (increment)** | Function that adds 1 to count |
| **Conditional Edge** | Routes to "continue" or "stop" |
| **Router Function** | `should_continue` - decision logic |
| **Loop** | Graph cycles back to increment node |
| **END** | Special node that terminates execution |

### 🎯 Flow Diagram

```
┌─────────────────────────────────────────┐
│ START with count = 0                    │
└──────────────┬──────────────────────────┘
               ↓
      ┌────────────────┐
      │   increment    │ ← Loop back if count < 5
      │  count += 1    │
      └───────┬────────┘
              ↓
      ┌───────────────┐
      │ should_continue│
      │   (router)     │
      └───┬───────┬───┘
          │       │
   count<5│       │count>=5
   "continue"    "stop"
          │       │
          ↓       ↓
        LOOP     END
```

### 💡 Real-World Applications

This pattern applies to:

- **Retry Logic** - Keep trying until success
- **Iterative Refinement** - Improve output until quality threshold met
- **Data Processing** - Process batches until all items handled
- **Polling** - Check status repeatedly until condition met
- **Multi-turn Conversations** - Continue dialogue until user exits

---

## 📖 Next Steps

Now that you understand basic StateGraph, explore:

1. **Multiple Nodes** - Build workflows with different processing stages
2. **Complex State** - Add more fields and data types
3. **Tool Integration** - Connect external APIs and services
4. **Checkpointing** - Save and resume graph state
5. **Human-in-the-Loop** - Add approval steps